In [1]:
import torch
from pytorch_quantization import tensor_quant

from torch import nn, outer

from pytorch_quantization import tensor_quant
import pytorch_quantization.nn as quant_nn
import pytorch_quantization
pytorch_quantization.__version__
from tqdm import tqdm


In [2]:
from datasets.yolo import YOLO
from engine.detector import Detector
from model.squeezedet import SqueezeDet, SqueezeDetWithLoss
from utils.config import Config
from utils.model import load_model
from PIL import Image
import numpy as np
import argparse
import os
from utils.misc import init_env

In [3]:
args = {'_': '_'}
cfg = Config().parse(args)

init_env(cfg) 
 
cfg.mode = 'eval'
cfg.exp_id = "test_veh_det_trt_calib"
cfg.dataset = 'yolo'
cfg.load_model = '../exp/exp5_traj_sq_with_mobilenetv2_3more_IRblocks_all_data/model_best.pth' 
cfg.batch_size = 1
cfg.arch = 'mobilenet_v2'
cfg.num_workers = 0
cfg.save_intervals = 1
cfg.stride = 8
cfg.debug = 0

def update_exp_dir(cfg, exp_id):
    cfg.save_dir = os.path.join(cfg.exp_dir, exp_id)
    os.makedirs(cfg.save_dir, exist_ok=True)
    cfg.log_file = os.path.join(cfg.save_dir, 'training_logs.txt')
    os.remove(cfg.log_file) if os.path.exists(cfg.log_file) else None
    cfg.debug_dir = os.path.join(cfg.save_dir, 'debug')

update_exp_dir(cfg, cfg.exp_id)

Config().print(cfg)


arch                           mobilenet_v2
batch_size                     1
bbox_loss_weight               20.0
chunk_sizes                    [32]
class_loss_weight              1.0
data_dir                       /media/gj_data/datasets
dataset                        yolo
debug                          0
debug_dir                      /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_veh_det_trt_calib/debug
device                         cuda
drift_prob                     1.0
dropout_prob                   0
exp_dir                        /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp
exp_id                         test_veh_det_trt_calib
flip_prob                      0.5
forbid_resize                  False
gpus                           [0]
gpus_str                       0
grad_norm                      0.5
keep_top_k                     64
load_model                     ../exp/exp5_traj_sq_with_mobilenetv2_3more_IRblocks_all_data/model_best.pth
load_pretrained      

In [4]:
# from eval import eval
# eval(cfg)

In [4]:
from engine.detector import Detector
from model.squeezedet import SqueezeDetWithLoss
from utils.model import load_model
from utils.misc import load_dataset



In [5]:
dataset = load_dataset(cfg.dataset)('val', cfg)
cfg = Config().update_dataset_info(cfg, dataset)
Config().print(cfg)


anchors                        [[  4.   4.  32.  20.]
 [  4.   4.  61.  42.]
 [  4.   4.  59.  97.]
 ...
 [444. 252. 160. 152.]
 [444. 252. 211. 201.]
 [444. 252. 343. 205.]]
anchors_per_grid               9
arch                           mobilenet_v2
batch_size                     1
bbox_loss_weight               20.0
chunk_sizes                    [32]
class_loss_weight              1.0
class_names                    ('bike', 'car', 'bus')
data_dir                       /media/gj_data/datasets
dataset                        yolo
debug                          0
debug_dir                      /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_veh_det_trt_calib/debug
device                         cuda
drift_prob                     1.0
dropout_prob                   0
exp_dir                        /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp
exp_id                         test_veh_det_trt_calib
flip_prob                      0.5
forbid_resize                  False
gpu

In [6]:
print (dataset.phase)

val


In [7]:
if cfg.mode=='eval':
        model = SqueezeDetWithLoss(cfg)
        model = load_model(model, cfg.load_model, cfg)


detect = model.detect
model.detect = True
detector = Detector(model, cfg)
print(detector.model)

loaded model ../exp/exp5_traj_sq_with_mobilenetv2_3more_IRblocks_all_data/model_best.pth, epoch 44
Model successfully loaded.
SqueezeDetWithLoss(
  (base): SqueezeDetBase(
    (quant): QuantStub()
    (dequant): DeQuantStub()
    (features): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
       

In [13]:
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib
from pytorch_quantization.tensor_quant import QuantDescriptor

from absl import logging
logging.set_verbosity(logging.INFO)  # Disable logging as they are too noisy in notebook

In [14]:
quant_desc_input = QuantDescriptor(calib_method='histogram')
quant_nn.QuantConv2d.set_default_quant_desc_input(quant_desc_input)
quant_nn.QuantLinear.set_default_quant_desc_input(quant_desc_input)

In [15]:
from pytorch_quantization import quant_modules
quant_modules.initialize()

In [16]:
print(detector.model)

SqueezeDetWithLoss(
  (base): SqueezeDetBase(
    (quant): QuantStub()
    (dequant): DeQuantStub()
    (features): Sequential(
      (0): ConvBNReLU(
        (0): QuantConv2d(
          3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
          (_input_quantizer): TensorQuantizer(8bit fake per-tensor amax=dynamic calibrator=HistogramCalibrator scale=1.0 quant)
          (_weight_quantizer): TensorQuantizer(8bit fake axis=0 amax=dynamic calibrator=MaxCalibrator scale=1.0 quant)
        )
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): QuantConv2d(
              32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False
              (_input_quantizer): TensorQuantizer(8bit fake per-tensor amax=dynamic calibrator=HistogramCalibrator scale=1.0 quant)
              (_weight_qu

### Calibration Functions

In [17]:
def collect_stats(model, data_loader, num_batches):
    """Feed data to the network and collect statistic"""

    # Enable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.disable_quant()
                module.enable_calib()
            else:
                module.disable()

    # for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
    #     model(image.cuda())
    #     if i >= num_batches:
    #         break
    # since yolo/lpr dataloader return a dict, so writing it in this way
    for i, (batch) in tqdm(enumerate(data_loader), total=num_batches):
        for k in batch:
                if 'image_meta' not in k:
                    batch[k] = batch[k].to(device=cfg.device, non_blocking=True)
        model(batch)
        if i >= num_batches:
            break

    # Disable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.enable_quant()
                module.disable_calib()
            else:
                module.enable()
            
def compute_amax(model, **kwargs):
    # Load calib result
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                if isinstance(module._calibrator, calib.MaxCalibrator):
                    module.load_calib_amax()
                else:
                    module.load_calib_amax(**kwargs)
#             print(F"{name:40}: {module}")
    model.cuda()

## Make Dataset and DataLoader

In [18]:
import os
import subprocess

import numpy as np
import skimage.io

from datasets.base import BaseDataset
from utils.boxes import generate_anchors
from PIL import Image
from torchvision.datasets.folder import default_loader


class YOLO_CalibTRT(BaseDataset):
    def __init__(self, phase, cfg):
        super(YOLO_CalibTRT, self).__init__(phase, cfg)

        self.input_size = (256, 448)  # (height, width), both dividable by 16
        self.class_names = ('bike', 'car', 'bus')
        # real_filtered mean and std
        # self.rgb_mean = np.array([94.87347, 96.89165, 94.70493], dtype=np.float32).reshape(1, 1, 3)
        # self.rgb_std = np.array([53.869507, 53.936283, 55.2807], dtype=np.float32).reshape(1, 1, 3)
        
        # real_filtered plus all_sites_seatbelt mean and std
        self.rgb_mean = np.array([104.90631, 105.41336, 104.70162], dtype=np.float32).reshape(1, 1, 3)
        self.rgb_std = np.array([50.69564, 49.60443, 50.158844], dtype=np.float32).reshape(1, 1, 3)
        self.num_classes = len(self.class_names)
        self.class_ids_dict = {cls_name: cls_id for cls_id, cls_name in enumerate(self.class_names)}

        self.data_dir = os.path.join(cfg.data_dir, 'all_real_plus_synth_8sites_plus_SVsynth_plus_seatbelt_plus_new_trajectory_data_kitti_format_5percentofwidth_filtered')
        self.sample_ids, self.sample_set_path = self.get_sample_ids()

        self.grid_size = tuple(x //cfg.stride  for x in self.input_size)  # anchors grid 
        # self.anchors_seed = np.array([[ 29, 17], [46, 32], [69, 52],
        #                                 [109, 68], [84, 127], [155, 106], 
        #                                 [255, 145], [183, 215], [371, 221]], dtype=np.float32) ## real_filtered anchors
        
        self.anchors_seed = np.array( [[ 32, 20], [ 61, 42], [ 59, 97],
                                        [103, 66], [122, 114], [183, 96],
                                        [160, 152], [211, 201], [343, 205]], dtype=np.float32) ## real_filtered plus all_sites_seatbelt anchors

        self.anchors = generate_anchors(self.grid_size, self.input_size, self.anchors_seed)
        self.anchors_per_grid = self.anchors_seed.shape[0]
        self.num_anchors = self.anchors.shape[0]

        self.results_dir = os.path.join(cfg.save_dir, 'results')

    def get_sample_ids(self):
        #a dirty duct tape to load preprocessing of val phase but load dataset for train phase for trt calib
        sample_set_name = 'train.txt' if self.phase == 'train' \
            else 'train.txt' if self.phase == 'val' \
            else 'trainval.txt' if self.phase == 'trainval' \
            else None

        sample_ids_path = os.path.join(self.data_dir, 'image_sets', sample_set_name)
        with open(sample_ids_path, 'r') as fp:
            sample_ids = fp.readlines()
        sample_ids = tuple(x.strip() for x in sample_ids)

        return sample_ids, sample_ids_path

    def load_image(self, index):
        image_id = self.sample_ids[index]
        image_path = os.path.join(self.data_dir, 'training/image_2', image_id + '.jpg')
        image = default_loader(image_path)
        if image.mode == 'L':
            image = image.convert('RGB')
        image = np.array(image).astype(np.float32)
        # image = skimage.io.imread(image_path).astype(np.float32)
        return image, image_id

    def load_annotations(self, index):
        ann_id = self.sample_ids[index]
        ann_path = os.path.join(self.data_dir, 'training/label_2', ann_id + '.txt')
        with open(ann_path, 'r') as fp:
            annotations = fp.readlines()

        annotations = [ann.strip().split(' ') for ann in annotations]
        class_ids, boxes = [], []
        for ann in annotations:
            if ann[0] not in self.class_names:
                continue
            class_ids.append(self.class_ids_dict[ann[0]])
            box = [float(x) for x in ann[4:8]]
            # if box[2] <= 0:
            #     box[2] = 0.00001
            # if box[3] <= 0:
            #     box[3] = 0.00001
            boxes.append(box)

        class_ids = np.array(class_ids, dtype=np.int16)
        boxes = np.array(boxes, dtype=np.float32)
        if len(boxes):
            return class_ids, boxes
        boxes = None
        return class_ids, boxes

    # ========================================
    #                evaluation
    # ========================================

    def save_results(self, results):
        txt_dir = os.path.join(self.results_dir, 'data')
        os.makedirs(txt_dir, exist_ok=True)

        for res in results:
            txt_path = os.path.join(txt_dir, res['image_meta']['image_id'] + '.txt')
            if 'class_ids' not in res:
                with open(txt_path, 'w') as fp:
                    fp.write('')
                continue

            num_boxes = len(res['class_ids'])
            with open(txt_path, 'w') as fp:
                for i in range(num_boxes):
                    class_name = self.class_names[res['class_ids'][i]].lower()
                    score = res['scores'][i]
                    bbox = res['boxes'][i, :]
                    line = '{} -1 -1 0 {:.2f} {:.2f} {:.2f} {:.2f} 0 0 0 0 0 0 0 {:.3f}\n'.format(
                            class_name, *bbox, score)
                    fp.write(line)

    def evaluate(self):
        kitti_eval_tool_path = os.path.join(self.cfg.root_dir, 'src/utils/kitti-eval/cpp/evaluate_object')
        cmd = '{} {} {} {} {}'.format(kitti_eval_tool_path,
                                      os.path.join(self.data_dir, 'training'),
                                      self.sample_set_path,
                                      self.results_dir,
                                      len(self.sample_ids))

        status = subprocess.call(cmd, shell=True)

        aps = {}
        for class_name in self.class_names:
            map_path = os.path.join(self.results_dir, 'stats_{}_ap.txt'.format(class_name.lower()))
            if os.path.exists(map_path):
                with open(map_path, 'r') as f:
                    lines = f.readlines()
                _aps = [float(line.split('=')[1].strip()) for line in lines]
            else:
                _aps = [0., 0., 0.]

            aps[class_name + '_easy'] = _aps[0]
            aps[class_name + '_moderate'] = _aps[1]
            aps[class_name + '_hard'] = _aps[2]

        aps['mAP'] = sum(aps.values()) / len(aps)

        return aps



In [19]:
veh_det_calib_dataset = YOLO_CalibTRT('val', cfg)

In [20]:
calib_data_loader = torch.utils.data.DataLoader(veh_det_calib_dataset,
                                                  batch_size=cfg.batch_size,
                                                  num_workers=cfg.num_workers,
                                                  pin_memory=False)

In [21]:
# print (len(calib_data_loader))
# inp  = next(iter(calib_data_loader))
# print(inp['image'].shape)

# for i, inp in enumerate(calib_data_loader, 0):
#         print(inp['image'].shape)
#         if i >= 10:
#             break

In [22]:
num_batches = int(len(veh_det_calib_dataset)/cfg.batch_size)
print (num_batches)
detector.model.eval()
detector.model.cuda()
with torch.no_grad():
    collect_stats(detector.model, calib_data_loader, num_batches=90000)
    # compute_amax(detector.model, method="percentile", percentile=99.99)

I0815 14:48:31.602073 140279568199872 tensor_quantizer.py:183] Disable `quant` stage.
I0815 14:48:31.602469 140279568199872 tensor_quantizer.py:179] Enable HistogramCalibrator
I0815 14:48:31.602790 140279568199872 tensor_quantizer.py:183] Disable `quant` stage.
I0815 14:48:31.603070 140279568199872 tensor_quantizer.py:179] Enable MaxCalibrator
I0815 14:48:31.603579 140279568199872 tensor_quantizer.py:183] Disable `quant` stage.
I0815 14:48:31.603878 140279568199872 tensor_quantizer.py:179] Enable HistogramCalibrator
I0815 14:48:31.604161 140279568199872 tensor_quantizer.py:183] Disable `quant` stage.
I0815 14:48:31.604446 140279568199872 tensor_quantizer.py:179] Enable MaxCalibrator
I0815 14:48:31.604721 140279568199872 tensor_quantizer.py:183] Disable `quant` stage.
I0815 14:48:31.604993 140279568199872 tensor_quantizer.py:179] Enable HistogramCalibrator
I0815 14:48:31.605275 140279568199872 tensor_quantizer.py:183] Disable `quant` stage.
I0815 14:48:31.605566 140279568199872 tensor_q

178655


  0%|          | 0/90000 [00:00<?, ?it/s]I0815 14:48:31.795305 140279568199872 histogram.py:69] Calibrator encountered negative values. It shouldn't happen after ReLU. Make sure this is the right tensor to calibrate.
I0815 14:48:31.802578 140279568199872 max.py:60] Calibrator encountered negative values. It shouldn't happen after ReLU. Make sure this is the right tensor to calibrate.
  0%|          | 106/90000 [01:26<20:16:24,  1.23it/s]


KeyboardInterrupt: 

In [ ]:
print (detector.model)

### Try Different Scehmes and Eval Model

In [54]:
from datasets.yolo import YOLO

eval_dataset = YOLO('val', cfg)
eval_dataset.phase

'val'

In [55]:
with torch.no_grad():
    for method in ["percentile", "mse", "entropy"]:
        percentile = 99.99
        print(F" ############  {method} calibration ######################")
        if(method == 'percentile'):
            compute_amax(detector.model, method=method, percentile = percentile)
        else:
            compute_amax(detector.model, method=method)
        update_exp_dir(cfg, f"trt_calib_veh_det_{method}")
        results = detector.detect_dataset(eval_dataset, cfg)
        eval_dataset.save_results(results)
        aps = eval_dataset.evaluate()
        for k, v in aps.items():
                print('{:<20} {:.3f}'.format(k, v))
        torch.save(detector.model.state_dict(), os.path.join(cfg.save_dir, f"quant_veh_det_squeezedet_{method}.pth"))

W0813 22:26:18.437037 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0813 22:26:18.437908 140513235423424 tensor_quantizer.py:238] Call .cuda() if running on GPU after loading calibrated amax.
W0813 22:26:18.438462 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0813 22:26:18.439009 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0813 22:26:18.439319 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0813 22:26:18.439835 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0813 22:26:18.440207 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([16, 1, 1, 1]).
W0813 22:26:18.440666 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0813 22:26:18.440975 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=to

 ############  percentile calibration ######################
eval: [0/7902] | data 0.100s | net 0.040s
eval: [20/7902] | data 0.072s | net 0.024s
eval: [40/7902] | data 0.100s | net 0.025s
eval: [60/7902] | data 0.046s | net 0.023s
eval: [80/7902] | data 0.085s | net 0.023s
eval: [100/7902] | data 0.084s | net 0.022s
eval: [120/7902] | data 0.083s | net 0.022s
eval: [140/7902] | data 0.081s | net 0.022s
eval: [160/7902] | data 0.093s | net 0.022s
eval: [180/7902] | data 0.078s | net 0.023s
eval: [200/7902] | data 0.110s | net 0.024s
eval: [220/7902] | data 0.100s | net 0.023s
eval: [240/7902] | data 0.098s | net 0.024s
eval: [260/7902] | data 0.078s | net 0.023s
eval: [280/7902] | data 0.079s | net 0.023s
eval: [300/7902] | data 0.083s | net 0.023s
eval: [320/7902] | data 0.106s | net 0.023s
eval: [340/7902] | data 0.083s | net 0.023s
eval: [360/7902] | data 0.079s | net 0.023s
eval: [380/7902] | data 0.080s | net 0.023s
eval: [400/7902] | data 0.094s | net 0.024s
eval: [420/7902] | da

sh: 1: gnuplot: not found
sh: 1: gnuplot: not found
Error: /undefinedfilename in (bike_detection.eps)
Operand stack:

Execution stack:
   %interp_exit   .runexec2   --nostringval--   --nostringval--   --nostringval--   2   %stopped_push   --nostringval--   --nostringval--   --nostringval--   false   1   %stopped_push
Dictionary stack:
   --dict:956/1684(ro)(G)--   --dict:0/20(G)--   --dict:77/200(L)--
Current allocation mode is local
Last OS error: No such file or directory
GPL Ghostscript 9.26: Unrecoverable error, exit code 1
sh: 1: pdfcrop: not found
sh: 1: gnuplot: not found
sh: 1: gnuplot: not found
Error: /undefinedfilename in (bike_orientation.eps)
Operand stack:

Execution stack:
   %interp_exit   .runexec2   --nostringval--   --nostringval--   --nostringval--   2   %stopped_push   --nostringval--   --nostringval--   --nostringval--   false   1   %stopped_push
Dictionary stack:
   --dict:956/1684(ro)(G)--   --dict:0/20(G)--   --dict:77/200(L)--
Current allocation mode is local


Thank you for participating in our evaluation!
Loading detections...
  done.
Your evaluation results are available in /home/gj/hazen/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_veh_det_trt_calib/results
bike_easy            0.290
bike_moderate        0.291
bike_hard            0.291
car_easy             0.734
car_moderate         0.729
car_hard             0.729
bus_easy             0.385
bus_moderate         0.384
bus_hard             0.384
mAP                  0.469
 ############  mse calibration ######################


W0813 22:41:01.803816 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0813 22:41:01.804579 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0813 22:41:02.576927 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0813 22:41:02.577594 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0813 22:41:03.639779 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0813 22:41:03.640609 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([16, 1, 1, 1]).
W0813 22:41:04.583522 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0813 22:41:04.584186 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([96, 1, 1, 1]).
W0813 22:41:05.551393 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Siz

...

W0813 22:59:46.658454 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0813 22:59:46.659085 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0813 22:59:51.492108 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0813 22:59:51.492670 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0813 23:00:00.013319 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0813 23:00:00.013832 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([16, 1, 1, 1]).
W0813 23:00:06.607147 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0813 23:00:06.607678 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([96, 1, 1, 1]).
W0813 23:00:14.147572 140513235423424 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Siz

eval: [0/7902] | data 0.112s | net 0.025s
eval: [20/7902] | data 0.117s | net 0.024s
eval: [40/7902] | data 0.076s | net 0.024s
eval: [60/7902] | data 0.077s | net 0.023s
eval: [80/7902] | data 0.112s | net 0.023s
eval: [100/7902] | data 0.119s | net 0.023s
eval: [120/7902] | data 0.130s | net 0.024s
eval: [140/7902] | data 0.132s | net 0.022s
eval: [160/7902] | data 0.159s | net 0.023s
eval: [180/7902] | data 0.118s | net 0.023s
eval: [200/7902] | data 0.178s | net 0.022s
eval: [220/7902] | data 0.133s | net 0.023s
eval: [240/7902] | data 0.110s | net 0.023s
eval: [260/7902] | data 0.132s | net 0.022s
eval: [280/7902] | data 0.133s | net 0.023s
eval: [300/7902] | data 0.137s | net 0.023s
eval: [320/7902] | data 0.131s | net 0.022s
eval: [340/7902] | data 0.121s | net 0.022s
eval: [360/7902] | data 0.209s | net 0.023s
eval: [380/7902] | data 0.138s | net 0.023s
eval: [400/7902] | data 0.136s | net 0.022s
eval: [420/7902] | data 0.159s | net 0.023s
eval: [440/7902] | data 0.127s | net 0

mkdir: cannot create directory ‘/home/gj/hazen/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_veh_det_trt_calib/results/plot’: File exists
sh: 1: gnuplot: not found
sh: 1: gnuplot: not found
Error: /undefinedfilename in (bike_detection.eps)
Operand stack:

Execution stack:
   %interp_exit   .runexec2   --nostringval--   --nostringval--   --nostringval--   2   %stopped_push   --nostringval--   --nostringval--   --nostringval--   false   1   %stopped_push
Dictionary stack:
   --dict:956/1684(ro)(G)--   --dict:0/20(G)--   --dict:77/200(L)--
Current allocation mode is local
Last OS error: No such file or directory
GPL Ghostscript 9.26: Unrecoverable error, exit code 1
sh: 1: pdfcrop: not found
sh: 1: gnuplot: not found
sh: 1: gnuplot: not found
Error: /undefinedfilename in (bike_orientation.eps)
Operand stack:

Execution stack:
   %interp_exit   .runexec2   --nostringval--   --nostringval--   --nostringval--   2   %stopped_push   --nostringval--   --nostringval--   --nostringval--   false   1 

Thank you for participating in our evaluation!
Loading detections...
  done.
Your evaluation results are available in /home/gj/hazen/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_veh_det_trt_calib/results
bike_easy            0.273
bike_moderate        0.272
bike_hard            0.272
car_easy             0.740
car_moderate         0.736
car_hard             0.736
bus_easy             0.391
bus_moderate         0.391
bus_hard             0.391
mAP                  0.467


sh: 1: gnuplot: not found
sh: 1: gnuplot: not found
Error: /undefinedfilename in (bus_detection.eps)
Operand stack:

Execution stack:
   %interp_exit   .runexec2   --nostringval--   --nostringval--   --nostringval--   2   %stopped_push   --nostringval--   --nostringval--   --nostringval--   false   1   %stopped_push
Dictionary stack:
   --dict:956/1684(ro)(G)--   --dict:0/20(G)--   --dict:77/200(L)--
Current allocation mode is local
Last OS error: No such file or directory
GPL Ghostscript 9.26: Unrecoverable error, exit code 1
sh: 1: pdfcrop: not found
sh: 1: gnuplot: not found
sh: 1: gnuplot: not found
Error: /undefinedfilename in (bus_orientation.eps)
Operand stack:

Execution stack:
   %interp_exit   .runexec2   --nostringval--   --nostringval--   --nostringval--   2   %stopped_push   --nostringval--   --nostringval--   --nostringval--   false   1   %stopped_push
Dictionary stack:
   --dict:956/1684(ro)(G)--   --dict:0/20(G)--   --dict:77/200(L)--
Current allocation mode is local
La

'val'

In [ ]:
# # print (detector.model)
# print (detector.model.base.features[0][0].input_quantizer.amax)
# print (detector.model.base.features[0][0].weight_quantizer.amax)

In [ ]:
# eval_dataset.save_results(results)
# aps = eval_dataset.evaluate()
# for k, v in aps.items():
#         print('{:<20} {:.3f}'.format(k, v))

In [177]:
# torch.save(detector.model.state_dict(), "../exp/quant_lp_squeezedet.pth")

In [ ]:
# test = detector.model.base
# test.cpu()
# quant_nn.TensorQuantizer.use_fb_fake_quant = True
# dummy_input = torch.randn(1, 3, 128, 128)

# # input_names = [ "network.0" ]
# # output_names = [ "network.23" ]

# # enable_onnx_checker needs to be disabled. See notes below.
# torch.onnx.export(
#     test, dummy_input, "../exp/test.onnx", verbose=True, opset_version=13, enable_onnx_checker=False)